In [1]:
!pip install -U ibm-watson-machine-learning


In [2]:
import getpass

wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",  
    "apikey": getpass.getpass("Enter your IBM Cloud API Key: ")
}


Enter your IBM Cloud API Key:  ········


In [3]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your Watson Studio Project ID: ")


In [4]:
import ibm_boto3
from botocore.client import Config
import types
import pandas as pd

def _iter_(self): return 0

cos_client = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id='QxCyWvbyiaf7ZlCstYDwbSO2k88J27qyby12NWabhzIm',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url="https://s3.us-south.cloud-object-storage.appdomain.cloud"  
)

bucket = 'bucket-p73ncjho1bgvuyh'
object_key = 'college_feedback_expanded.csv' 

body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
if not hasattr(body, "_iter"): body._iter = types.MethodType(_iter_, body)

df = pd.read_csv(body)
df.head()


,Feedback,True_Category
0,The classrooms are too small and the fans don'...,Facilities
1,The professors explain concepts very clearly.,Academics
2,The admin office delays issuing important docu...,Administration
3,There are not enough projectors in the seminar...,Facilities
4,The timetable is not well-structured.,Academics


In [5]:
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams


In [6]:
model_id = ModelTypes.FLAN_T5_XXL  
parameters = {
    GenParams.MAX_NEW_TOKENS: 10,
    GenParams.DECODING_METHOD: "greedy"
}

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=wml_credentials,
    project_id=project_id
)

model.get_details()


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:273: LifecycleWarning: Model 'google/flan-t5-xxl' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


{'model_id': 'google/flan-t5-xxl',
 'label': 'flan-t5-xxl-11b',
 'provider': 'Google',
 'source': 'Hugging Face',
 'functions': [{'id': 'text_generation'}],
 'short_description': 'flan-t5-xxl is an 11 billion parameter model based on the Flan-T5 family.',
 'long_description': 'flan-t5-xxl (11B) is an 11 billion parameter model based on the Flan-T5 family. It is a pretrained T5 - an encoder-decoder model pre-trained on a mixture of supervised / unsupervised tasks converted into a text-to-text format, and fine-tuned on the Fine-tuned Language Net (FLAN) with instructions for better zero-shot and few-shot performance.',
 'terms_url': 'https://huggingface.co/google/flan-t5-xxl/blob/main/README.md',
 'input_tier': 'class_2',
 'output_tier': 'class_2',
 'number_params': '11b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'

In [7]:
few_shot_prompt = """
Classify the following student feedback into one of these categories: Academics, Facilities, or Administration.

Example 1:
Feedback: "The computer labs do not have updated software."
Category: Facilities

Example 2:
Feedback: "Professors are often not available after class."
Category: Academics

Example 3:
Feedback: "It takes too long to get fee receipts processed."
Category: Administration
"""


In [11]:
results = []
feedbacks = df["Feedback"].tolist()

for fb in feedbacks:
    prompt = f"{few_shot_prompt}\nFeedback: \"{fb}\"\nCategory:"
    response = model.generate(prompt=prompt) 
    results.append(response['results'][0]['generated_text'])


In [12]:
df["Predicted_Category"] = results
df[["Feedback", "Predicted_Category"]].head()


,Feedback,Predicted_Category
0,The classrooms are too small and the fans don'...,Facilities
1,The professors explain concepts very clearly.,Academics
2,The admin office delays issuing important docu...,Administration
3,There are not enough projectors in the seminar...,Facilities
4,The timetable is not well-structured.,Academics


In [20]:
df.to_csv("/home/wsuser/work/college_feedback_with_predictions.csv", index=False)


In [22]:
import os
os.listdir("/home/wsuser/work/")


['college_feedback_with_predictions.csv']

In [23]:
with open('/home/wsuser/work/college_feedback_with_predictions.csv', 'r') as f:
    content = f.read()
print(content)


Feedback,True_Category,Predicted_Category
The classrooms are too small and the fans don't work.,Facilities,Facilities
The professors explain concepts very clearly.,Academics,Academics
The admin office delays issuing important documents.,Administration,Administration
There are not enough projectors in the seminar halls.,Facilities,Facilities
The timetable is not well-structured.,Academics,Academics
The Wi-Fi is always down in the hostel.,Facilities,Facilities
The exam results are not uploaded on time.,Administration,Administration
The library is well-maintained and has all the required books.,Academics,Facilities
Too much administrative paperwork for fee payment.,Administration,Administration
The faculty is supportive during lab sessions.,Academics,Academics
The cafeteria is unhygienic and food quality is poor.,Facilities,Facilities
The internal marks are not properly communicated.,Academics,Academics
There is no proper parking space for students.,Facilities,Facilities
The online portal